## **이미지 색상 반전**

In [19]:
import cv2
import numpy as np
import os
from glob import  glob
import shutil

dataset_path = r"C:\Users\USER\Desktop\현우\Augment data + 직접 원거리\Augmented Data"
output_path = r"C:\Users\USER\Desktop\현우\Augment data + 직접 원거리\색상 반전 사진"

sets = ['train', 'valid', 'test']

def process_image(image_path, output_path):
    image = cv2.imread(image_path)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    #흰색 범위 마스크 생성
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask_white = cv2.inRange(hsv, lower_white, upper_white)

    #분홍색 범위 마스크 생성
    lower_pink = np.array([140, 50, 100])
    upper_pink = np.array([170, 255, 255])
    mask_pink = cv2.inRange(hsv, lower_pink, upper_pink)

    #흰색과 분홍색 마스크 결합 (or 연산)
    mask_combined = cv2.bitwise_or(mask_white, mask_pink)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    background = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    result = np.where(mask_combined[:, :, None] == 255, image, background)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, result)

for dataset in sets:
    input_image_folder = os.path.join(dataset_path, dataset, 'images')
    input_label_folder = os.path.join(dataset_path, dataset, 'labels')
    
    output_image_folder = os.path.join(output_path, dataset, 'images')
    output_label_folder = os.path.join(output_path, dataset, 'labels')

    image_paths = []
    for ext in ('*.jpg', '*.jpeg', '*.png'):
        image_paths.extend(glob(os.path.join(input_image_folder, ext)))

    for img_path in image_paths:
        img_name = os.path.basename(img_path)
        output_image_path = os.path.join(output_image_folder, img_name)
        process_image(img_path, output_image_path)

    if not os.path.exists(input_label_folder):
        print(f'{input_label_folder} 폴더가 없습니다. 라벨 복사 생략')
        continue

    os.makedirs(output_label_folder, exist_ok=True)
    label_paths = glob(os.path.join(input_label_folder, "*.txt"))

    for label_path in label_paths:
        label_name = os.path.basename(label_path)
        output_label_path = os.path.join(output_label_folder, label_name)
        shutil.copy(label_path, output_label_path)

print('모든 데이터셋 변환 완료')

모든 데이터셋 변환 완료


## **yaml 생성**

In [8]:
import yaml

data = {
    'train' : r"C:\Users\USER\Desktop\현우\Augment data + 직접 원거리\색상 반전 사진(train, valid, test)\train",
    'val' : r"C:\Users\USER\Desktop\현우\Augment data + 직접 원거리\색상 반전 사진(train, valid, test)\valid",
    'test' : r"C:\Users\USER\Desktop\현우\Augment data + 직접 원거리\색상 반전 사진(train, valid, test)\test",
    'names' : {0 : 'flower', 1 : 'bud'},
    'nc' : 2
}

with open('./flower.yaml', 'w') as f:
    yaml.dump(data, f)

with open('./flower.yaml', 'r') as f:
    lines = yaml.safe_load(f)
    print(lines)

{'names': {0: 'flower', 1: 'bud'}, 'nc': 2, 'test': 'C:\\Users\\USER\\Desktop\\현우\\Augment data + 직접 원거리\\색상 반전 사진(train, valid, test)\\test', 'train': 'C:\\Users\\USER\\Desktop\\현우\\Augment data + 직접 원거리\\색상 반전 사진(train, valid, test)\\train', 'val': 'C:\\Users\\USER\\Desktop\\현우\\Augment data + 직접 원거리\\색상 반전 사진(train, valid, test)\\valid'}


## **모델 훈련**

In [9]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

model.train(
    data=r"C:\Users\USER\Desktop\현우\flower.yaml",
    epochs=150,
    batch=16,
    imgsz=1280,
    augment=False,
    pretrained=True,
    patience=20,
    hsv_h=0,
    hsv_s=0,
    hsv_v=0,
    flipud=0,
    fliplr=0
)

Ultralytics 8.3.39  Python-3.11.9 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## **새로운 이미지에 대해 색상 반전**

In [ ]:
import cv2
import numpy as np
import os
from glob import glob

def preprocess_new_image(image_path, output_path):
    """새로운 이미지에서도 배경을 회색(gray)으로 변환하고, 흰색/분홍색을 강조"""
    image = cv2.imread(image_path)

    # RGB → HSV 변환
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 흰색 및 분홍색 범위 설정
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    mask_white = cv2.inRange(hsv, lower_white, upper_white)

    lower_pink = np.array([140, 50, 100])
    upper_pink = np.array([170, 255, 255])
    mask_pink = cv2.inRange(hsv, lower_pink, upper_pink)

    # 두 개의 마스크를 합침
    mask_combined = cv2.bitwise_or(mask_white, mask_pink)

    # 배경을 흑백으로 변환
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    background = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # 흰색 및 분홍색을 유지하고, 나머지는 회색으로 변환
    result = np.where(mask_combined[:, :, None] == 255, image, background)

    # 변환된 이미지 저장
    cv2.imwrite(output_path, result)

# 폴더 내 모든 이미지 변환 함수
def preprocess_folder(input_folder, output_folder):
    """폴더 내 모든 이미지 변환"""
    os.makedirs(output_folder, exist_ok=True)  # 변환된 이미지 저장 폴더 생성

    # JPG, JPEG, PNG 파일 리스트 가져오기
    image_paths = []
    for ext in ("*.jpg", "*.jpeg", "*.png"):
        image_paths.extend(glob(os.path.join(input_folder, ext)))

    # 이미지 변환 적용
    for image_path in image_paths:
        img_name = os.path.basename(image_path)  # 파일명만 추출
        output_path = os.path.join(output_folder, img_name)  # 변환된 이미지 저장 경로
        preprocess_new_image(image_path, output_path)

    print(f"폴더 변환 완료: {input_folder} → {output_folder}")

# (폴더 경로 지정)
input_folder = r"C:\Users\USER\Desktop\현우\Inference Data"  # 원본 이미지 폴더
output_folder = r"C:\Users\USER\Desktop\현우\Inference Data\Inference 색상 변환"  # 변환된 이미지 저장 폴더

preprocess_folder(input_folder, output_folder)


폴더 변환 완료: C:\Users\USER\Desktop\현우\Inference Data → C:\Users\USER\Desktop\현우\Inference Data\Inference 색상 변환


## **새로운 이미지에 대한 예측**

In [ ]:
from ultralytics import YOLO

model = YOLO(r"C:\Users\USER\Desktop\현우\runs\detect\train5\weights\best.pt")

results = model.predict(r"C:\Users\USER\Desktop\현우\Inference Data\Inference 색상 변환", save=True, imgsz=1280)



image 1/9 C:\Users\USER\Desktop\\Inference Data\Inference  \20240412_131120.jpg: 736x1280 23 flowers, 43.4ms
image 2/9 C:\Users\USER\Desktop\\Inference Data\Inference  \20240412_131535.jpg: 1280x736 6 flowers, 35.0ms
image 3/9 C:\Users\USER\Desktop\\Inference Data\Inference  \20240412_131736.jpg: 736x1280 5 flowers, 27.9ms
image 4/9 C:\Users\USER\Desktop\\Inference Data\Inference  \20240412_132609.jpg: 1280x736 9 flowers, 29.2ms
image 5/9 C:\Users\USER\Desktop\\Inference Data\Inference  \360_F_887268339_lu8fEPhF5rUoK54mow2P66DD2xpaeJh6.jpg: 736x1280 1 flower, 8 buds, 26.4ms
image 6/9 C:\Users\USER\Desktop\\Inference Data\Inference  \Apple-tree-bloom.jpg: 864x1280 4 flowers, 41 buds, 39.7ms
image 7/9 C:\Users\USER\Desktop\\Inference Data\Inference  \IMG_0890-1024x768.jpg: 960x1280 8 flowers, 2 buds, 54.6ms
image 8/9 C:\Users\USER\Desktop\\Inference Data\Inference  \KakaoTalk_20240418_130907386_16.jpg: 1280x960 34 flowers, 42.3ms
image 9/9 C:\Users\USER\Desktop\\Inference Data\Inference